In [1]:
import csv
import os
from time import time

import numpy as np
from nltk.tokenize import TweetTokenizer
from sklearn import metrics as skmetrics
import torch
torch.backends.cudnn.enabled = False

use_segmentation = False
embedding_dim = 25
embedding_file = 'data/glove.twitter.27B/glove.twitter.27B.%dd.txt' % embedding_dim
olid_data = 'data/OLIDv1.0/olid-training-v1.0.tsv'
olid_hashtags = 'data/olid_segmentations.tsv'
if torch.cuda.is_available():
    device = 0 
    print('CUDA available! Using device %d (%s)' % (device, torch.cuda.get_device_name(device)))
else:
    device = None
    print('CUDA unavailable! Using CPU.')

np.random.seed(1234) # helps reproducibility

CUDA available! Using device 0 (GeForce GTX 1070)


In [2]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    # https://discuss.pytorch.org/t/is-there-something-like-keras-utils-to-categorical-in-pytorch/5960
    return np.eye(num_classes, dtype='uint8')[y]

def report(y, y_hat, metrics=['accuracy', 'precision', 'recall', 'f1-weighted', 'f1-macro']):
    results = []
    metrics = metrics.copy()
    while len(metrics) > 0:
        m = metrics.pop(0)
        if m == 'accuracy':
            results.append(skmetrics.accuracy_score(y, y_hat))
        elif m == 'precision':
            results.append(skmetrics.precision_score(y, y_hat))
        elif m == 'recall':
            results.append(skmetrics.recall_score(y, y_hat))
        elif m == 'f1-weighted':
            results.append(skmetrics.f1_score(y, y_hat, average='weighted'))
        elif m == 'f1-macro':
            results.append(skmetrics.f1_score(y, y_hat, average='macro'))
        else:
            print('Metric unknown: %s' % m)
    return results

In [3]:
start = time()
# Load tweets and labels
with open(olid_data, encoding='utf-8') as f:
    raw = csv.reader(f, delimiter='\t')
    x_raw = []
    y = []
    for r in raw:
        x_raw.append(r[1])
        y.append(0 if r[2] == 'NOT' else 1)
    x_raw = x_raw[1:]
    y = np.array(y[1:])
    bad_words = [row[:-1] for row in f.readlines()[1:]]
    
# Load hashtag segmentations
segmentations = {}
for line in open(olid_hashtags, encoding='utf-8'):
    terms = [x.strip() for x in line.split('\t')]
    hashtag, segmentation = terms[0], terms[1]
    segmentations[hashtag] = segmentation

# Tokenize data
tokenizer = TweetTokenizer(preserve_case=False)  
x = []
vocab = {}
i = 1 # start from 1. 0 is pad.
for tweet in x_raw:
    example = []
    for token in tokenizer.tokenize(tweet):
        # if it's a hashtag, look up segmentaion
        if use_segmentation and token[0] == '#' and token[1:] in segmentations:
            sequence = segmentations[token[1:]].split()
        else:
            sequence = [token]
            
        for word in sequence:
            if word not in vocab:
                vocab[word] = i
                i += 1
            example.append(vocab[word])
    x.append(example)
    
# Randomly shuffle
# As the result of a 3 hour long bug hunt,
# we have to subtract 1 because cudnn throws a fit if it tries to access
# the example at the end of the file. Some illegal memory access error?
# This is wack and I'm mad.
i = np.arange(len(x) - 1)  
np.random.shuffle(i)
x = [torch.LongTensor(x[k]).to(device) for k in i]
y = torch.FloatTensor(to_categorical(y[i], 2)).to(device)

split = 0.7
split_index = int(len(x) * split)
x_train = x[:split_index]
y_train = y[:split_index]
x_val = x[split_index:]
y_val = y[split_index:]
print('Loaded data in %.2fs' % (time() - start))

Loaded data in 3.52s


In [4]:
# Load embeddings
# On my mac, GloVe loads 25D in 30s, 50D in 100s, 100D in 630s
start = time()
embeddings = {}
with open(embedding_file, encoding='utf-8') as f:
    raw = [row.split() for row in f.readlines()]
    for r in raw:
        embeddings[r[0]] = np.array([float(v) for v in r[1:]])

# Create embedding weight matrix that corresponds to the ids we've already set for x
# https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76
embedding_weights = np.zeros((len(vocab) + 1, embedding_dim)) # add 1 to account for pad
for word, i in vocab.items():
    try: 
        embedding_weights[i] = embeddings[word]
    except KeyError:
        embedding_weights[i] = np.random.normal(scale=0.6, size=(embedding_dim,))
embedding_weights = torch.FloatTensor(embedding_weights)
print('Loaded embeddings in %.2fs' % (time() - start)) 

Loaded embeddings in 15.22s


In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.utils.rnn as rnn

class GRU(nn.Module):
    def __init__(self, vocab_size, embeddings=None, dim_emb=25, n_classes=2, device=0):
        super(GRU, self).__init__()
        self.vocab_size = vocab_size
        self.n_classes = n_classes
        self.device = device
        
        gru_size = 128
        fc_size = 1024
        if embeddings is None:
            self.dim_emb = dim_emb
            self.embedding = nn.Embedding(self.vocab_size, self.dim_emb)
        else:
            self.dim_emb = embeddings.shape[1]
            self.embedding = nn.Embedding.from_pretrained(embeddings) # defaults to frozen
        self.gru = nn.GRU(input_size=self.dim_emb,
                          hidden_size=gru_size,
                          num_layers=2,
                          batch_first=True,
                          bidirectional=True
                         )
        self.fc1 = nn.Linear(2 * gru_size, self.n_classes) #fc_size)
        self.fc2 = nn.Linear(fc_size, fc_size)
        self.fc3 = nn.Linear(fc_size, self.n_classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)

    def forward(self, x):
        x = self.embedding(x)
        out, h = self.gru(x)
        x = h.permute(1, 0, 2)
        x = torch.flatten(x, 1)
        x = self.relu(out[:, -1])
        # x = self.relu(self.fc1(x))
        # x = self.relu(self.fc2(x))
        y = self.softmax(self.fc1(x))
        return y
    
    def predict(self, x, one_hot=False, batch_size=32):
        if one_hot:
            y = torch.zeros((len(x), self.n_classes))
        else:
            y = torch.zeros((len(x)))
        for i in range(0, len(x), batch_size):
            batch = rnn.pack_sequence(x[i:i+batch_size], enforce_sorted=False)
            batch, _ = rnn.pad_packed_sequence(batch, batch_first=True)
            logits = self.forward(batch)
            preds = torch.max(logits, 1)[1]
            if one_hot:
                pass # broken
                # y[i:i+batch_size, preds] = 1
            else:
                y[i:i+batch_size] = preds
        return y
    
def train(x_train, y_train, x_val, y_val, vocab_size, batch_size, lr, epochs):
    print('Start Training!')
    mlp = GRU(vocab_size, embedding_weights)
    if torch.cuda.is_available():
        mlp.cuda()
    optimizer = optim.Adam(mlp.parameters(), lr=lr)
    max_f1 = 0
    print('epoch time  t_loss   t_acc  v_acc  t_f1m  v_f1m  v_f1w')
    for epoch in range(epochs):
        start = time()
        total_loss = 0.0
        for i in range(0, len(x_train), batch_size):
            mlp.zero_grad()
            batch = rnn.pack_sequence(x_train[i:i+batch_size], enforce_sorted=False)
            batch, _ = rnn.pad_packed_sequence(batch, batch_first=True)
            probs = mlp.forward(batch)
            NLL = torch.neg(torch.log(probs))
            y_batch = y_train[i:i+batch_size]
            losses = torch.bmm(NLL.view(len(y_batch), 1, 2), y_batch.view(len(y_batch), 2, 1))
            loss = torch.sum(losses) / len(y_batch)
            total_loss += loss
            loss.backward()
            optimizer.step()
        metrics = ['accuracy', 'f1-weighted', 'f1-macro']
        t_y_hat = mlp.predict(x_train, False, 1024)
        t_dense = torch.max(y_train, 1)[1]
        t_acc, t_f1w, t_f1m = report(t_dense.cpu(), t_y_hat, metrics=metrics)
        v_y_hat = mlp.predict(x_val, False, 1024)
        v_dense = torch.max(y_val, 1)[1]
        v_acc, v_f1w, v_f1m = report(v_dense.cpu(), v_y_hat, metrics=metrics)
        print('\r%-5d %.2fs %08.4f %.4f %.4f %.4f %.4f %.4f' % 
              (epoch, time() - start, total_loss, t_acc, v_acc, t_f1m, v_f1m, v_f1w), end='')
        if v_f1m > max_f1:
             print('\tNew best macro f1! Saving model.')
             torch.save(mlp.state_dict(), 'models/best-bigru-f1%d.model' % epoch)
             max_f1 = v_f1m
    return mlp

In [ ]:
# helpful for quick debugging. [0, len(x_train)) means no limit
lower = 0
upper = len(x_train)
batch_size = 128
lr = 3e-4
epochs = 10000
mlp = train(x_train[lower:upper], y_train[lower:upper], x_val, y_val, len(vocab), batch_size, lr, epochs)

Start Training!
epoch time  t_loss   t_acc  v_acc  t_f1m  v_f1m  v_f1w
0     8.87s 116.2137 0.3358 0.3311 0.2563 0.2501 0.1668	New best macro f1! Saving model.
2     8.61s 116.2036 0.3777 0.3313 0.3464 0.2503 0.1669	New best macro f1! Saving model.
7     8.39s 116.1938 0.4170 0.3318 0.4106 0.2511 0.1679	New best macro f1! Saving model.
8     8.45s 116.1900 0.4504 0.3338 0.4504 0.2566 0.1756	New best macro f1! Saving model.
10    8.36s 116.1816 0.4799 0.3361 0.4757 0.2617 0.1824	New best macro f1! Saving model.
11    8.41s 116.1822 0.5371 0.4068 0.5049 0.3973 0.3716	New best macro f1! Saving model.
13    8.31s 116.1742 0.5618 0.4678 0.5124 0.4650 0.4780	New best macro f1! Saving model.
14    8.28s 116.1451 0.6295 0.6259 0.5511 0.5494 0.6122	New best macro f1! Saving model.
15    8.37s 115.9445 0.6136 0.6062 0.5726 0.5657 0.6106	New best macro f1! Saving model.
16    8.36s 115.6773 0.6477 0.6382 0.6223 0.6114 0.6459	New best macro f1! Saving model.
17    8.30s 115.0728 0.6962 0.6908 0.67